In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plpt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler

In [2]:
def replace_negative_values_with_median(df):
  cols_with_negative_values = [col for col in df.columns if (df[col] < 0).any()]
  for col in cols_with_negative_values:
    median = df[col].median()
    df[col] = df[col].apply(lambda x: round(median) if x < 0 else x)
  return df

In [3]:
df1 = pd.read_csv('../Dataset/dataset_1.csv',encoding='utf-8', engine='python')

In [4]:
df1.head()

,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,qty_space_url,...,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
0,3,0,0,1,0,0,0,0,0,0,...,1,2,0,892,0,0,0,0,0,1
1,5,0,1,3,0,3,0,2,0,0,...,1,2,1,9540,1,0,0,0,0,1
2,2,0,0,1,0,0,0,0,0,0,...,1,2,3,589,1,0,0,0,0,0
3,4,0,2,5,0,0,0,0,0,0,...,1,2,0,292,1,0,0,0,0,1
4,2,0,0,0,0,0,0,0,0,0,...,1,2,1,3597,0,1,0,0,0,0


In [5]:
# removing all the column that has same values in all the rows
nunique = df1.nunique()
cols_to_drop = nunique[nunique == 1].index
df1 = df1.drop(cols_to_drop, axis=1)

In [6]:
df1 = replace_negative_values_with_median(df1)

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88647 entries, 0 to 88646
Data columns (total 99 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   qty_dot_url                 88647 non-null  int64  
 1   qty_hyphen_url              88647 non-null  int64  
 2   qty_underline_url           88647 non-null  int64  
 3   qty_slash_url               88647 non-null  int64  
 4   qty_questionmark_url        88647 non-null  int64  
 5   qty_equal_url               88647 non-null  int64  
 6   qty_at_url                  88647 non-null  int64  
 7   qty_and_url                 88647 non-null  int64  
 8   qty_exclamation_url         88647 non-null  int64  
 9   qty_space_url               88647 non-null  int64  
 10  qty_tilde_url               88647 non-null  int64  
 11  qty_comma_url               88647 non-null  int64  
 12  qty_plus_url                88647 non-null  int64  
 13  qty_asterisk_url            886

In [20]:
df2 = pd.read_csv('../Dataset/dataset_2.csv',encoding='utf-8', engine='python')

In [21]:
df2.head()

,url,url_length,hostname_length,ip,total_of.,total_of-,total_of@,total_of?,total_of&,total_of=,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.progarchives.com/album.asp?id=61737,46,20,0,3,0,0,1,0,1,...,1,1,0,627,6678,78526,0,0,5,phishing
1,http://signin.eday.co.uk.ws.edayisapi.dllsign....,128,120,0,10,0,0,0,0,0,...,1,0,0,300,65,0,0,1,0,phishing
2,http://www.avevaconstruction.com/blesstool/ima...,52,25,0,3,0,0,0,0,0,...,1,0,0,119,1707,0,0,1,0,phishing
3,http://www.jp519.com/,21,13,0,2,0,0,0,0,0,...,1,1,0,130,1331,0,0,0,0,legitimate
4,https://www.velocidr1.com/,28,19,0,2,0,0,0,0,0,...,0,0,0,164,1662,312044,0,0,4,legitimate


In [22]:
# removing all the column that has same values in all the rows
nunique = df2.nunique()
cols_to_drop = nunique[nunique == 1].index
df2 = df2.drop(cols_to_drop, axis=1)

In [23]:
df2 = df2.drop(labels=['url'], axis=1)
df2['status'] = df2['status'].map({'phishing': 1, 'legitimate': 0})

In [24]:
df2 = replace_negative_values_with_median(df2)

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19431 entries, 0 to 19430
Data columns (total 80 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url_length                  19431 non-null  int64  
 1   hostname_length             19431 non-null  int64  
 2   ip                          19431 non-null  int64  
 3   total_of.                   19431 non-null  int64  
 4   total_of-                   19431 non-null  int64  
 5   total_of@                   19431 non-null  int64  
 6   total_of?                   19431 non-null  int64  
 7   total_of&                   19431 non-null  int64  
 8   total_of=                   19431 non-null  int64  
 9   total_of_                   19431 non-null  int64  
 10  total_of~                   19431 non-null  int64  
 11  total_of%                   19431 non-null  int64  
 12  total_of/                   19431 non-null  int64  
 13  total_of*                   194

In [28]:
# this function is only for dataset 1
sm = SMOTE(random_state=42)
rus = RandomUnderSampler(random_state=42)
sm_enn = SMOTEENN(random_state=42)
def different_train_test_1(partition_name,dataset_idx,df,ratio):
    train, test = train_test_split(df, test_size=ratio)
    test.to_csv('../Dataset/Testing/Dataset'+dataset_idx+'/'+partition_name+'_test.csv', index = None)
    X_train_oversampled, Y_train_oversampled = sm.fit_resample(train.drop(labels=[train.columns[-1]], axis=1),train[train.columns[-1]])
    X_train_oversampled['phishing'] = Y_train_oversampled
    X_train_oversampled.to_csv('../Dataset/Training/Dataset'+dataset_idx+'/Oversampled/'+partition_name+'_train.csv', index = None)
    print(partition_name,'Oversampled '+'training dataset'+dataset_idx+'\n',X_train_oversampled[X_train_oversampled.columns[-1]].value_counts())

    X_train_undersampled, Y_train_undersampled = rus.fit_resample(train.drop(labels=[train.columns[-1]], axis=1),train[train.columns[-1]])
    X_train_undersampled['phishing'] = Y_train_undersampled
    X_train_undersampled.to_csv('../Dataset/Training/Dataset'+dataset_idx+'/Undersampled/'+partition_name+'_train.csv', index = None)
    print(partition_name,'Undersampled '+'training dataset'+dataset_idx+'\n',X_train_undersampled[X_train_undersampled.columns[-1]].value_counts())

    X_train_over_undersampled, Y_train_over_undersampled = sm_enn.fit_resample(train.drop(labels=[train.columns[-1]], axis=1),train[train.columns[-1]])
    X_train_over_undersampled['phishing'] = Y_train_over_undersampled
    X_train_over_undersampled.to_csv('../Dataset/Training/Dataset'+dataset_idx+'/Over_Undersampled/'+partition_name+'_train.csv', index = None)
    print(partition_name,'Over_Undersampled '+'training dataset'+dataset_idx+'\n',X_train_over_undersampled[X_train_over_undersampled.columns[-1]].value_counts())

In [29]:
different_train_test_1('80_20','1',df1,0.2)
different_train_test_1('75_25','1',df1,0.25)
different_train_test_1('70_30','1',df1,0.3)
different_train_test_1('65_35','1',df1,0.35)
different_train_test_1('60_40','1',df1,0.4)

80_20 Oversampled training dataset1
 phishing
0    46492
1    46492
Name: count, dtype: int64
80_20 Undersampled training dataset1
 phishing
0    24425
1    24425
Name: count, dtype: int64
80_20 Over_Undersampled training dataset1
 phishing
1    38539
0    37006
Name: count, dtype: int64
75_25 Oversampled training dataset1
 phishing
1    43622
0    43622
Name: count, dtype: int64
75_25 Undersampled training dataset1
 phishing
0    22863
1    22863
Name: count, dtype: int64
75_25 Over_Undersampled training dataset1
 phishing
1    36174
0    34561
Name: count, dtype: int64
70_30 Oversampled training dataset1
 phishing
1    40614
0    40614
Name: count, dtype: int64
70_30 Undersampled training dataset1
 phishing
0    21438
1    21438
Name: count, dtype: int64
70_30 Over_Undersampled training dataset1
 phishing
1    33500
0    32079
Name: count, dtype: int64
65_35 Oversampled training dataset1
 phishing
0    37813
1    37813
Name: count, dtype: int64
65_35 Undersampled training dataset1
 p

In [30]:
# this function will  work for any normal and balanced dataset
def different_train_test_2(partition_name,dataset_idx,df,ratio):
    train, test = train_test_split(df, test_size=ratio)
    test.to_csv('../Dataset/Testing/Dataset'+dataset_idx+'/'+partition_name+'_test.csv', index = None)
    train.to_csv('../Dataset/Training/Dataset'+dataset_idx+'/'+partition_name+'_train.csv', index = None)

In [32]:
different_train_test_2('80_20','2',df2,0.2)
different_train_test_2('75_25','2',df2,0.25)
different_train_test_2('70_30','2',df2,0.3)
different_train_test_2('65_35','2',df2,0.35)
different_train_test_2('60_40','2',df2,0.4)